In [1]:
import config
from IPython.display import IFrame
from random import randint
import pandas as pd
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import warnings
warnings.filterwarnings('ignore')
import pickle
from tabulate import tabulate

In [2]:
df_final = pd.read_csv("export_recommender.csv")

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [4]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [5]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!") 


In [6]:
scaler = load("scaler.pickle")
kmeans = load("kmeans_4.pickle")

In [7]:
#Input User Song
def user_input():
    Input = input("Please enter an artist or song: ")
    results = sp.search(q= Input,limit=3,market="GB")
    global id
    id=results["tracks"]["items"][0]["id"]
    return play_song(id)
     

In [8]:
def recommender():
    features = sp.audio_features(id)
    user_df=pd.DataFrame(features)    
    user_df=user_df[["danceability","energy","key", "loudness","acousticness", "valence", "tempo"]]
    user_key = user_df["key"].values
    user_temp = user_df["tempo"].values
    user_scaled = scaler.transform(user_df)
    user_clusters = kmeans.predict(user_scaled)
    recommendation_df = df_final[df_final["cluster"]==user_clusters[0]] 
    record1_df = recommendation_df[recommendation_df["key"]==user_key[0]]
    record_df = record1_df[record1_df["tempo"].between(user_temp[0]-10, user_temp[0]+10)].sample(n=1)
    printed_r_df=record_df[["danceability","energy","key", "loudness","acousticness", "valence", "tempo"]]
    r_id = record_df["id"].values
    print("These are the features of the song you chose: ")
    print(tabulate(user_df, headers='keys', tablefmt='psql'))
    print("These are the features of my recommendation: ")
    print(tabulate(printed_r_df, headers="keys", tablefmt='psql'))
    print("__________________________________________________________________")
    print("This is the song I recommend to you, best regards your friendly AI: ")
    return play_song(r_id[0])

In [9]:
user_input()


Please enter an artist or song: micheal jackson


In [10]:
recommender()

These are the features of the song you chose: 
+----+----------------+----------+-------+------------+----------------+-----------+---------+
|    |   danceability |   energy |   key |   loudness |   acousticness |   valence |   tempo |
|----+----------------+----------+-------+------------+----------------+-----------+---------|
|  0 |          0.545 |    0.485 |     9 |     -8.045 |          0.537 |     0.101 |  80.948 |
+----+----------------+----------+-------+------------+----------------+-----------+---------+
These are the features of my recommendation: 
+------+----------------+----------+-------+------------+----------------+-----------+---------+
|      |   danceability |   energy |   key |   loudness |   acousticness |   valence |   tempo |
|------+----------------+----------+-------+------------+----------------+-----------+---------|
| 9038 |          0.469 |     0.42 |     9 |     -7.824 |          0.701 |     0.232 |  88.016 |
+------+----------------+----------+-------+

In [11]:
    features = sp.audio_features(id)
    user_df=pd.DataFrame(features)    
    user_df=user_df[["danceability","energy","key", "loudness","acousticness", "valence", "tempo"]]
    user_key = user_df["key"].values
    user_temp = user_df["tempo"].values
    user_scaled = scaler.transform(user_df)
    user_clusters = kmeans.predict(user_scaled)
    recommendation_df = df_final[df_final["cluster"]==user_clusters[0]] 
    record1_df = recommendation_df[recommendation_df["key"]==user_key[0]]
    record_df = record1_df[record1_df["tempo"].between(user_temp[0]-10, user_temp[0]+10)]
record_df

,id,danceability,energy,key,loudness,acousticness,valence,tempo,cluster
1363,4anx9dy5q5vU1B0kwNRmrR,0.560,0.501,9,-5.442,0.725,0.220,90.301,38
1512,0EvnRzwVhtaboVkVSYvxyA,0.451,0.547,9,-8.522,0.504,0.126,87.721,38
4593,7woW97CfcWaKtuC6W5BP2K,0.545,0.485,9,-8.045,0.537,0.101,80.948,38
5126,5lvnGiTAH9BVlJLHc0DXZz,0.439,0.322,9,-5.959,0.692,0.160,77.963,38
8035,4DXfOXYv0ki7IR3P0XRcMY,0.594,0.459,9,-7.507,0.712,0.316,75.082,38
8878,5O2lEvI1K4Fa4KL8vO3e6W,0.228,0.431,9,-9.465,0.261,0.206,83.066,38
9038,2v6AWRzhAsy61rBNBqzEna,0.469,0.420,9,-7.824,0.701,0.232,88.016,38
9683,3AH9lQx7COmyQKq5SqJRrX,0.383,0.429,9,-10.236,0.580,0.204,83.807,38
10680,2ewjgFhHDTE8BkV8kzpiX2,0.305,0.628,9,-3.573,0.460,0.428,80.389,38
